# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |

## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.
* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.
* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

You can find the best combinations of these parameters with [bench_speculative.py](https://github.com/sgl-project/sglang/blob/main/scripts/playground/bench_speculative.py).

In the documentation below, we set `--cuda-graph-max-bs` to be a small value for faster engine startup. For your own workloads, please tune the above parameters together with `--cuda-graph-max-bs`, `--max-running-requests`, `--mem-fraction-static` for the best performance. 

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative-algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

[2025-12-12 05:23:43] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2025-12-12 05:23:43] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2025-12-12 05:23:43] INFO utils.py:164: NumExpr defaulting to 16 threads.


In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8 --log-level warning
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-12-12 05:23:50] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-12-12 05:23:50] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-12-12 05:23:50] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-12-12 05:23:53] WARNING server_args.py:1416: Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-12-12 05:23:53] WARNING server_args.py:1844: Overlap scheduler is disabled because of using eagle3 or standalone speculative decoding.


[2025-12-12 05:24:01] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-12-12 05:24:01] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-12-12 05:24:01] INFO utils.py:164: NumExpr defaulting to 16 threads.
[2025-12-12 05:24:01] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-12-12 05:24:01] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-12-12 05:24:01] INFO utils.py:164: NumExpr defaulting to 16 threads.


[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
I1212 05:24:04.743409 836092 transfer_engine.cpp:486] Metrics reporting is disabled (set MC_TE_METRIC=1 to enable)
I1212 05:24:04.743430 836092 transfer_engine.cpp:91] Transfer Engine parseHostNameWithPort. server_name: 10.184.97.146 port: 12001
I1212 05:24:04.743456 836092 transfer_engine.cpp:146] Transfer Engine RPC using P2P handshake, listening on 10.184.97.146:15797
I1212 05:24:04.743525 836092 transfer_engine.cpp:185] Auto-discovering topology...
I1212 05:24:04.746387 836092 transfer_engine.cpp:200] Topology discovery complete. Found 9 HCAs.
I1212 05:24:04.771056 836092 rdma_context.cpp:533] Find best gid index: 3 on mlx5_

I1212 05:24:04.940001 836092 rdma_context.cpp:533] Find best gid index: 3 on mlx5_roce2/
I1212 05:24:04.940827 836092 rdma_context.cpp:126] RDMA device: mlx5_roce2, LID: 0, GID: (GID_Index 3) 00:00:00:00:00:00:00:00:00:00:ff:ff:16:49:31:5d
I1212 05:24:04.945062 836092 rdma_context.cpp:533] Find best gid index: 3 on mlx5_roce3/
I1212 05:24:04.945799 836092 rdma_context.cpp:126] RDMA device: mlx5_roce3, LID: 0, GID: (GID_Index 3) 00:00:00:00:00:00:00:00:00:00:ff:ff:16:49:b1:5d
I1212 05:24:04.949944 836092 rdma_context.cpp:533] Find best gid index: 3 on mlx5_roce4/
I1212 05:24:04.950675 836092 rdma_context.cpp:126] RDMA device: mlx5_roce4, LID: 0, GID: (GID_Index 3) 00:00:00:00:00:00:00:00:00:00:ff:ff:16:4a:31:5d
I1212 05:24:04.954841 836092 rdma_context.cpp:533] Find best gid index: 3 on mlx5_roce5/
I1212 05:24:04.955612 836092 rdma_context.cpp:126] RDMA device: mlx5_roce5, LID: 0, GID: (GID_Index 3) 00:00:00:00:00:00:00:00:00:00:ff:ff:16:4a:b1:5d
I1212 05:24:04.959796 836092 rdma_co

...

Capturing batches (bs=2 avail_mem=37.21 GB):  50%|█████     | 2/4 [00:05<00:04,  2.23s/it]

Capturing batches (bs=1 avail_mem=37.19 GB):  75%|███████▌  | 3/4 [00:05<00:01,  1.36s/it]

Capturing batches (bs=1 avail_mem=37.19 GB): 100%|██████████| 4/4 [00:08<00:00,  2.24s/it]


Capturing batches (bs=1 avail_mem=37.09 GB): 100%|██████████| 4/4 [00:00<00:00, 48.05it/s]


[2025-12-12 05:24:28] Endpoint '/get_model_info' is deprecated and will be removed in a future version. Please use '/model_info' instead.


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

In [4]:
terminate_process(server_process)

### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2 --log-level warning
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-12-12 05:24:38] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-12-12 05:24:38] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-12-12 05:24:38] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-12-12 05:24:41] WARNING server_args.py:1416: Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-12-12 05:24:41] WARNING server_args.py:1844: Overlap scheduler is disabled because of using eagle3 or standalone speculative decoding.


[2025-12-12 05:24:47] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-12-12 05:24:47] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-12-12 05:24:47] INFO utils.py:164: NumExpr defaulting to 16 threads.
[2025-12-12 05:24:47] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-12-12 05:24:47] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-12-12 05:24:47] INFO utils.py:164: NumExpr defaulting to 16 threads.


[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
I1212 05:24:49.746986 839339 transfer_engine.cpp:486] Metrics reporting is disabled (set MC_TE_METRIC=1 to enable)
I1212 05:24:49.747002 839339 transfer_engine.cpp:91] Transfer Engine parseHostNameWithPort. server_name: 10.184.97.146 port: 12001
I1212 05:24:49.747026 839339 transfer_engine.cpp:146] Transfer Engine RPC using P2P handshake, listening on 10.184.97.146:16533
I1212 05:24:49.747110 839339 transfer_engine.cpp:185] Auto-discovering topology...
I1212 05:24:49.750032 839339 transfer_engine.cpp:200] Topology discovery complete. Found 9 HCAs.
I1212 05:24:49.775020 839339 rdma_context.cpp:533] Find best gid index: 3 on mlx5_

I1212 05:24:49.943980 839339 rdma_context.cpp:533] Find best gid index: 3 on mlx5_roce2/
I1212 05:24:49.944864 839339 rdma_context.cpp:126] RDMA device: mlx5_roce2, LID: 0, GID: (GID_Index 3) 00:00:00:00:00:00:00:00:00:00:ff:ff:16:49:31:5d
I1212 05:24:49.970952 839339 rdma_context.cpp:533] Find best gid index: 3 on mlx5_roce3/
I1212 05:24:49.971801 839339 rdma_context.cpp:126] RDMA device: mlx5_roce3, LID: 0, GID: (GID_Index 3) 00:00:00:00:00:00:00:00:00:00:ff:ff:16:49:b1:5d
I1212 05:24:49.998997 839339 rdma_context.cpp:533] Find best gid index: 3 on mlx5_roce4/
I1212 05:24:49.999836 839339 rdma_context.cpp:126] RDMA device: mlx5_roce4, LID: 0, GID: (GID_Index 3) 00:00:00:00:00:00:00:00:00:00:ff:ff:16:4a:31:5d
I1212 05:24:50.027009 839339 rdma_context.cpp:533] Find best gid index: 3 on mlx5_roce5/
I1212 05:24:50.027823 839339 rdma_context.cpp:126] RDMA device: mlx5_roce5, LID: 0, GID: (GID_Index 3) 00:00:00:00:00:00:00:00:00:00:ff:ff:16:4a:b1:5d
I1212 05:24:50.054971 839339 rdma_co

...

Capturing batches (bs=1 avail_mem=37.38 GB):  75%|███████▌  | 3/4 [00:06<00:01,  1.67s/it]

Capturing batches (bs=1 avail_mem=37.38 GB): 100%|██████████| 4/4 [00:11<00:00,  2.97s/it]


Capturing batches (bs=1 avail_mem=18.80 GB):  25%|██▌       | 1/4 [00:00<00:00,  6.29it/s]

Capturing batches (bs=1 avail_mem=18.80 GB): 100%|██████████| 4/4 [00:00<00:00, 19.38it/s]


[2025-12-12 05:26:04] Endpoint '/get_model_info' is deprecated and will be removed in a future version. Please use '/model_info' instead.


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

In [7]:
terminate_process(server_process)

### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16  --log-level warning
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-12-12 05:26:15] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-12-12 05:26:15] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-12-12 05:26:15] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-12-12 05:26:17] WARNING model_config.py:917: Casting torch.bfloat16 to torch.float16.
[2025-12-12 05:26:17] WARNING server_args.py:1416: Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-12-12 05:26:17] WARNING server_args.py:1844: Overlap scheduler is disabled because of using eagle3 or standalone speculative decoding.


[2025-12-12 05:26:17] Casting torch.bfloat16 to torch.float16.


[2025-12-12 05:26:27] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-12-12 05:26:27] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-12-12 05:26:27] INFO utils.py:164: NumExpr defaulting to 16 threads.
[2025-12-12 05:26:27] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-12-12 05:26:27] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-12-12 05:26:27] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-12-12 05:26:29] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=2 avail_mem=24.83 GB):  50%|█████     | 2/4 [00:05<00:04,  2.32s/it]

Capturing batches (bs=1 avail_mem=24.79 GB):  75%|███████▌  | 3/4 [00:05<00:01,  1.38s/it]

Capturing batches (bs=1 avail_mem=24.79 GB): 100%|██████████| 4/4 [00:08<00:00,  2.06s/it]


Capturing batches (bs=1 avail_mem=24.65 GB): 100%|██████████| 4/4 [00:00<00:00, 34.87it/s]


[2025-12-12 05:27:12] Endpoint '/get_model_info' is deprecated and will be removed in a future version. Please use '/model_info' instead.


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

In [10]:
terminate_process(server_process)

### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative-algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16 --log-level warning
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-12-12 05:27:25] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-12-12 05:27:25] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-12-12 05:27:25] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-12-12 05:27:27] WARNING model_config.py:917: Casting torch.bfloat16 to torch.float16.
[2025-12-12 05:27:27] WARNING server_args.py:1416: Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-12-12 05:27:27] WARNING server_args.py:1844: Overlap scheduler is disabled because of using eagle3 or standalone speculative decoding.


[2025-12-12 05:27:28] Casting torch.bfloat16 to torch.float16.


[2025-12-12 05:27:34] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-12-12 05:27:34] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-12-12 05:27:34] INFO utils.py:164: NumExpr defaulting to 16 threads.
[2025-12-12 05:27:34] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-12-12 05:27:34] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-12-12 05:27:34] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-12-12 05:27:36] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=2 avail_mem=25.87 GB):  50%|█████     | 2/4 [00:03<00:03,  1.51s/it]

Capturing batches (bs=1 avail_mem=25.82 GB):  75%|███████▌  | 3/4 [00:03<00:00,  1.04it/s]

Capturing batches (bs=1 avail_mem=25.82 GB): 100%|██████████| 4/4 [00:06<00:00,  1.51s/it]


Capturing batches (bs=1 avail_mem=25.67 GB): 100%|██████████| 4/4 [00:00<00:00, 32.52it/s]


[2025-12-12 05:28:11] Endpoint '/get_model_info' is deprecated and will be removed in a future version. Please use '/model_info' instead.


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

In [13]:
terminate_process(server_process)

## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../basic_usage/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5 --log-level warning
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-12-12 05:28:23] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-12-12 05:28:23] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-12-12 05:28:23] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-12-12 05:28:26] WARNING server_args.py:1416: Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-12-12 05:28:26] WARNING server_args.py:1844: Overlap scheduler is disabled because of using eagle3 or standalone speculative decoding.


[2025-12-12 05:28:32] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-12-12 05:28:32] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-12-12 05:28:32] INFO utils.py:164: NumExpr defaulting to 16 threads.
[2025-12-12 05:28:33] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-12-12 05:28:33] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-12-12 05:28:33] INFO utils.py:164: NumExpr defaulting to 16 threads.


[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
I1212 05:28:36.071269 851969 transfer_engine.cpp:486] Metrics reporting is disabled (set MC_TE_METRIC=1 to enable)
I1212 05:28:36.072510 851969 transfer_engine.cpp:91] Transfer Engine parseHostNameWithPort. server_name: 10.184.97.146 port: 12001
I1212 05:28:36.072574 851969 transfer_engine.cpp:146] Transfer Engine RPC using P2P handshake, listening on 10.184.97.146:15417
I1212 05:28:36.072650 851969 transfer_engine.cpp:185] Auto-discovering topology...
I1212 05:28:36.083628 851969 transfer_engine.cpp:200] Topology discovery complete. Found 9 HCAs.
I1212 05:28:36.090286 851969 rdma_context.cpp:533] Find best gid index: 3 on mlx5_

W1212 05:28:37.257460 851969 memory_location.cpp:72] Failed to get NUMA node, addr: 0x7f9b83fff010, len: 2147483648: Operation not permitted [1]


...

W1212 05:28:57.734529 851969 memory_location.cpp:72] Failed to get NUMA node, addr: 0x7f9447fff010, len: 2147483648: Operation not permitted [1]
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  4.06it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:00<00:00,  4.56it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:00<00:00,  4.52it/s]



Capturing batches (bs=1 avail_mem=7.72 GB): 100%|██████████| 4/4 [00:00<00:00, 59.46it/s]


[2025-12-12 05:29:01] Endpoint '/get_model_info' is deprecated and will be removed in a future version. Please use '/model_info' instead.


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

In [16]:
terminate_process(server_process)

## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).